In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import datetime

import matplotlib.pyplot as plt

from scipy import stats
import math
import random

from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit, train_test_split, cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import  mean_squared_log_error

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [ ]:
#Global Variables for grid search
n_splits = 10
n_jobs = 2

max_depth_min = 3
max_depth_max = 11
n_estimator_min = 100
n_estimator_max = 200

test_size =0.5
random_state = 1986

In [ ]:
train_File = '../input/train.csv'
test_File = '../input/test.csv'

df_train = pd.read_csv(train_File)
df_test = pd.read_csv(test_File)
df_test['SalePrice'] = 0
df_concat = pd.concat([df_train,df_test])
#df_concat.info()

In [ ]:
df_concat.head()

In [ ]:
def giveMeWrangledData(df, testFile=False, log=False):
    
    
    df = df.drop(['Id', 'GarageYrBlt','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF'],axis=1)
    
    df['LotFrontage'] =df.LotFrontage.fillna(df.LotFrontage.mode()[0])
    df['MasVnrArea']=df.MasVnrArea.fillna(0.0)
    df['TotalBsmtSF'] = df.TotalBsmtSF.fillna(0)
    df['BsmtFullBath'] = df.BsmtFullBath.fillna(0)
    df['BsmtHalfBath'] = df.BsmtHalfBath.fillna(0)
    df['GarageCars'] = df.GarageCars.fillna(0)
    df['GarageArea'] = df.GarageArea.fillna(0)
    
    #convert data type
    #we are being little lineant to give int64 for YearBuilt, YrSold but those guys are going to be box-coxed 
    #so let them at least enjoy the bigger size for now
    int64_variables = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', \
                     'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', \
                     'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',\
                     'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', \
                     'PoolArea', 'MiscVal', 'YrSold', 'SalePrice']
    
    #if testFile:
    #    int64_variables.remove('SalePrice')

        
        
        
    
    for c in int64_variables:
        if log:
            print("Changing the data type for :", c)
        #df[c] = df[c].astype(np.int64)
        df[c] = df[c].astype(np.float64)
        
    int_to_categorical_variables = ['MSSubClass', 'OverallQual', 'OverallCond', 'FireplaceQu', 'MoSold']
    for c in int_to_categorical_variables:
        df[c] = df[c].astype(str)
        
    df = df.fillna('NotAvailable')
    return df
df = giveMeWrangledData(df_concat)
df.head()

In [ ]:
#df.info()

In [ ]:
def preProcessData(df, log=False):
    
    

    #get dummies
    if log:
        print("Shape of the data set before pre processing : ", df.shape )
        print("Categorical columns : ", list(df.select_dtypes(exclude=np.number)))
    df = pd.get_dummies(df, dtype=np.float64)
    
    
    
    if log:
        print("\n\nShape of the data set after pre processing : ", df.shape )
        print("Columns in the data set are : ",list(df))

    return df

df_prep = preProcessData(df)
df_prep.info()

https://stats.stackexchange.com/questions/130262/why-not-log-transform-all-variables-that-are-not-of-main-interest

In [ ]:
from IPython import display
def transformTarget(df_temp, revert=False ):
    '''df = df_temp.copy()
    #df['new_variable'] = df.BedroomAbvGr * .1 + df.FullBath *.25 + df.HalfBath * .5 + df.BsmtFullBath *.75 + df.BsmtHalfBath * 1
    #df['new_variable'] = df.BedroomAbvGr * .1 + df.FullBath *.25 + df.HalfBath * .5 + df.BsmtFullBath *.75 + df.BsmtHalfBath * .1
    df['new_variable'] = df.BedroomAbvGr.apply(lambda x: x if x > 0 else 1)
    #print("new_variable calculated")
    #display.display(df[df.new_variable.isna()])
    
    if not revert:
        df['sales_per_new'] = df['SalePrice']/ df.new_variable
        #display.display(df[df.sales_per_new.isna()])
        y = np.array(df['sales_per_new'].apply( lambda x: math.log(x)))
        return y
    
    df['sales_per_new'] = df['SalePrice']* df.new_variable
    #print("sales_per_new calculated")
    #display.display(df[df.sales_per_new.isna()])'''
    return np.log( df_temp.SalePrice / df_temp.GrLivArea)
    #return np.log(df_temp['SalePrice'])
    

In [ ]:
def newBoxCoxTranformation(df_temp,target,testFile=False, log=False):
    df = df_temp.copy()
    #assuming that only numerical features are presented
    if log:
        print("Shape of the dataset initial : ", df.shape)
    
    if not testFile:
        df =df[df.SalePrice >0]
        if log:
            print("Shape of the dataset before transformation : ", df.shape)
        
        display.display(df[df['SalePrice'].isna()])
        y = transformTarget(df)
        X= df.drop([target],axis = 1)
        #X=df.drop(target, axis=1)
        x_columns = list(X)
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        X_testx = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X,y)
        X_testxx = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X,df.SalePrice)
        print(np.unique(X == X_testx))
        print(np.unique(X == X_testxx))
        print(np.unique(X_testx == X_testxx))
        X = pd.DataFrame(X, columns=x_columns)
        
        for c in list(X):
            if len(X[c].unique()) in  [1,2]:
                if log:
                    print("Skipping Transformation for ", c, "because unique values are :",X[c].unique())
            else:
                if log:
                    print("Boxcoxing : ", c)
                X[c] = stats.boxcox(X[c])[0]
        
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        #X = X.values
        if log:
            print("Shape of the dataset after transformation : ", X.shape, y.shape)
        return X,y
    else:
        df = df[df.SalePrice == 0.0]
        if log:
            print("Shape of the dataset before transformation : ", df.shape)
        X=df.drop(target,axis = 1)
        x_columns = list(X)
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        
        X = pd.DataFrame(X, columns=x_columns)
        for c in list(X):
            if len(X[c].unique()) in  [1,2]:
                if log:
                    print("Skipping Transformation for ", c, "because unique values are :",X[c].unique())
            else:
                if log:
                    print("Boxcoxing : ", c)
                X[c] = stats.boxcox(X[c])[0]
        
        
        #X = preprocessing.power_transform( X, method='box-cox')
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        #X = X.values
        if log:
            print("Shape of the dataset after transformation : ", X.shape)
        return X
        
    

X = newBoxCoxTranformation(df_prep,'SalePrice',True,False)  
X,y = newBoxCoxTranformation(df_prep,'SalePrice',False,False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=random.randint(1,500))#, stratify=df.BldgType)

In [ ]:
reg = XGBRegressor( colsample_bylevel=0.5,colsample_bytree=0.5, 
                   learning_rate=0.1, max_depth=3, 
       n_estimators=100, n_jobs=2, reg_alpha=0.5, reg_lambda=0.5)
reg.fit(X_train,y_train)
reg.score(X_test,y_test)

In [ ]:
np.sqrt(mean_squared_log_error(y_test, reg.predict(X_test)))

In [ ]:
np.sqrt(mean_squared_log_error(np.exp(y_test), np.exp(reg.predict(X_test))))

In [ ]:
reg.fit(X,y)

In [ ]:
reg

##### We need to have different pre-processing logic to test data. We will come back to it little later.

In [ ]:
def checkTheTestFile(reg, auto_ml=False):
    df_test = pd.read_csv(test_File)
    df_test['SalePrice'] = 0.0
    
    
    
    df_train =  pd.read_csv(train_File)
    df_concat = pd.concat([df_train,df_test])

    #print(df_test[df_test.TotalBsmtSF.isna()])
    #return
    df = giveMeWrangledData(df_concat,True)
    
    #print(df.info())
    df = preProcessData(df)
    X_columns = list(df.columns)
    X_columns.remove('SalePrice')
    #df_test_dummy = df.copy()
    #df_test_dummy = df_test_dummy[df_test_dummy.SalePrice == 0]
    #print(df.info())
    X = newBoxCoxTranformation(df,'SalePrice',True)
    
    if auto_ml:
        X = pd.DataFrame(X, columns= X_columns)
    #print(np.sqrt(mean_squared_log_error(y, reg.predict(X))))
    
    #df_test_dummy['SalePrice'] = np.exp(reg.predict(X))
    #display.display(df_test_dummy[df_test_dummy['SalePrice'].isna()])
    
    
    #df_test['SalePrice'] = np.exp(reg.predict(X))
    df_test['SalePrice_predicted'] = np.exp(reg.predict(X))
    df_test['SalePrice'] = df_test['SalePrice_predicted'] * df_test['GrLivArea']
    
    
    
    #display.display(df_test[df_test['SalePrice'].isna()])
    
    return df_test, X, reg.predict(X)
df_test, X_dummy, y_dummy= checkTheTestFile(reg)

In [ ]:
df_test.SalePrice.describe()

In [ ]:
df_test[df_test.SalePrice.isna()]

In [ ]:
df_test[df_test.SalePrice == 0]

In [ ]:
df_test[['Id','SalePrice']]

In [ ]:
df_test[['Id','SalePrice']].to_csv('submission.csv',index=False)

##### I got Kaggle Rank of 2539/4463 with RMSLE =0.14357
##### As on 1/17/2019 : 9:06PM IST
* 0.13501 ==> 2040 
* 0.13252 ==> 1865
* 0.13002 ==> 1704
* 0.12658 ==> 1500
* 0.12351 ==> 1250
* 0.12081 ==> 1000
* 0.11572 ==> 500
* 0.11475 ==> 250
* 0.11310 ==> 100
* 0.10985 ==> 50
* 0.10973 ==> 25
* 0.10845 ==> 10
* 0.08021 ==> 5
* 0.00000 ==> 1

In [ ]:
def forCrossValidationStratifiedShuffleSplit(df):
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    #print("Number of Splits configured :", sss.get_n_splits(df, df.BldgType))
    
    for train_index, test_index in sss.split(df, df.BldgType):
        yield train_index, test_index
        
    for train_index, test_index in sss.split(df, df.OverallQual):
        yield train_index, test_index

In [ ]:
def crossValidationScoring(reg,X,y):
    
    return -np.sqrt(mean_squared_log_error(np.exp(y), 
                                          np.exp(reg.predict(X))
                                          ))

    #return np.sqrt(mean_squared_log_error((y), 
    #                                      (reg.predict(X))
    #                                      ))
mean_temp_rmsle = np.mean(cross_val_score(reg,X,y,cv= 5,scoring='neg_mean_squared_log_error'))
print("RMSE with without target variable transformation : ", np.sqrt(mean_temp_rmsle * -1))

mean_temp_rmsle = np.mean(cross_val_score(reg, X, y,
                                          cv= forCrossValidationStratifiedShuffleSplit(df_train),
                                          scoring=crossValidationScoring))
print("RMSE with post target variable transformation : ", mean_temp_rmsle)

##### Below are the best scores I have run so far (Kaggle Scores)
XGBRegressor(max_depth=6,n_estimator=100, 
                   colsample_bytree=.25,
                   colsample_bylevel=1,
                   reg_alpha =.5,
                   reg_lambda=.5, 
                   learning_rate=0.1,                   
                  )
				  
reg = XGBRegressor(max_depth=6,n_estimator=100, 
                   colsample_bytree=.25,
                   colsample_bylevel=1,
                   reg_alpha =0.5,
                   reg_lambda=0, 
                   learning_rate=0.1,                   
                  )
				  
				  
reg = XGBRegressor(max_depth=6,n_estimator=100, 
                   colsample_bytree=.25,
                   colsample_bylevel=1,
                   reg_alpha =1,
                   reg_lambda=0, 
                   learning_rate=0.1,                   
                  )
				  
reg = XGBRegressor(max_depth=5,n_estimator=100, 
                   colsample_bylevel=.25,
                   colsample_bytree=0.5,
                   learning_rate=0.1,)
				   
reg =  XGBRegressor(max_depth=4,n_estimator=200)

In [ ]:
def checkTheNewBestOfBest():
    
    start_time = datetime.datetime.now()
    
    df_train = pd.read_csv(train_File)
    df_test = pd.read_csv(test_File)
    df_test['SalePrice'] = 0
    df_concat = pd.concat([df_train,df_test])
    
    df = giveMeWrangledData(df_concat)
    df_prep = preProcessData(df)
    
    X,y = newBoxCoxTranformation(df_prep,'SalePrice',False,False)
    
    def rmsle_cv(model, X,y):
        cv = ShuffleSplit(n_splits=3, test_size=random.randint(7,9)/10, random_state=random.randint(1,1000))
        cross_cv = cross_val_score(model, X, y, cv=cv, scoring='r2', n_jobs=3)
        return(cross_cv)

    xgb1 = XGBRegressor(max_depth=4,n_estimator=200)
    xgb2 = XGBRegressor(max_depth=5,n_estimator=100, 
                   colsample_bylevel=.25,
                   colsample_bytree=0.5,
                   learning_rate=0.1,)
    xgb3 = XGBRegressor(max_depth=6,n_estimator=100, 
                   colsample_bytree=.25,
                   colsample_bylevel=1,
                   reg_alpha =0.5,
                   reg_lambda=0, 
                   learning_rate=0.1,                   
                  )
    xgb4 = XGBRegressor(max_depth=6,n_estimator=100, 
                   colsample_bytree=.25,
                   colsample_bylevel=1,
                   reg_alpha =1,
                   reg_lambda=0, 
                   learning_rate=0.1,                   
                  )

    xgb5 = XGBRegressor(max_depth=6,n_estimator=100, 
                   colsample_bytree=.25,
                   colsample_bylevel=1,
                   reg_alpha =.5,
                   reg_lambda=.5, 
                   learning_rate=0.1,                   
                  )
    lr = LinearRegression()
    
    class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
        def __init__(self, base_models, meta_model, n_folds=5):
            self.base_models = base_models
            self.meta_model = meta_model
            self.n_folds = n_folds

        # We again fit the data on clones of the original models
        def fit(self, X, y):

            self.base_models_ = [list() for x in self.base_models]
            self.meta_model_ = clone(self.meta_model)
            kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=1986)

            # Train cloned base models then create out-of-fold predictions
            # that are needed to train the cloned meta-model
            out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
            for i, model in enumerate(self.base_models):
                for train_index, holdout_index in kfold.split(X, y):
                    instance = clone(model)
                    self.base_models_[i].append(instance)
                    instance.fit(X[train_index], y[train_index])
                    y_pred = instance.predict(X[holdout_index])
                    out_of_fold_predictions[holdout_index, i] = y_pred

            # Now train the cloned  meta-model using the out-of-fold predictions as new feature
            self.meta_model_.fit(out_of_fold_predictions, y)
            return self

        #Do the predictions of all base models on the test data and use the averaged predictions as 
        #meta-features for the final prediction which is done by the meta-model
        def predict(self, X):
            meta_features = np.column_stack([
                np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
                for base_models in self.base_models_ ])
            return self.meta_model_.predict(meta_features)


    stacked_averaged_models = StackingAveragedModels(base_models = (xgb1, xgb2, xgb3, xgb4, xgb5),
                                                     meta_model = lr)
    def rmsle(y, y_pred):
        return np.sqrt(mean_squared_error(y, y_pred))

    score = rmsle_cv(stacked_averaged_models, X, y)
    print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

    stacked_averaged_models.fit(X, y)
    df_test, X_dummy, y_dummy= checkTheTestFile(stacked_averaged_models)
    #df_test['pred_rent_per_bed'] = np.exp(stacked_averaged_models.predict(test))
    #df_test['pred_rent'] = df_test['pred_rent_per_bed'] * df_test['bed']

    #print("Test Score r2 (post inverse function) : ", r2_score(df_test.rent, df_test.pred_rent))
    
    #printStats(stacked_test_pred, y_test)
    #print("\n\nAs per Competetion Evaluation Metrics :")
    #printStats(df_test.pred_rent, df_test.rent)
    
    
    print("\n\nTotal time taken by the stack approach : ",datetime.datetime.now() - start_time )

    return stacked_averaged_models, df_test

stacked_xgboost_reg_new, df_test_new = checkTheNewBestOfBest()

from sklearn.model_selection import GridSearchCV
def gridSearchCVImp():
    start_time =datetime.datetime.now()
    reg = XGBRegressor(reg_alpha =.5, reg_lambda=.5, n_jobs =2)
    
    parameters = {
        'max-dpth':list(range(3,11)),
        'colsample_bylevel' : np.arange(0.0, 1.0, 0.25),
        'colsample_bytree' : np.arange(0.0, 1.0, 0.25)
        }
    #cv= ShuffleSplit(n_splits=20, test_size=.5, random_state=1986)
    reg_grid = GridSearchCV(reg, parameters, 
                            cv=forCrossValidationStratifiedShuffleSplit(df_train),
                            n_jobs = 2,
                            #scoring = 'neg_mean_absolute_error',
                            #scoring = 'neg_mean_squared_log_error',
                            scoring = crossValidationScoring,
                            verbose=1
                            #pre_dispatch = 2
                           )
    reg_grid.fit(X,y)
    
    print("Total time for the gridserach", datetime.datetime.now() - start_time)
    
    return reg_grid
reg_grid = gridSearchCVImp()
print(reg_grid.best_estimator_)
print(reg_grid.best_score_)

##### when GridSearchCV replaces exactly the code return in manually

* Total time for the gridserach 0:03:56.721226

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max-dpth=3,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=2, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.5, reg_lambda=0.5, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
       
-0.14418238953245838

##### run 2
* Total time for the gridserach 0:03:48.821054

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max-dpth=3,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=2, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.5, reg_lambda=0.5, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
       
-0.14418238953245838

##### run 3
* Total time for the gridserach 0:03:49.627057

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max-dpth=3,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=2, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.5, reg_lambda=0.5, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
       
-0.14418238953245838

In [ ]:
print(reg_grid.best_score_)

#for log target/livarea

#for target / livarea
def doGridSearch():
    
    start_time = datetime.datetime.now()
    
    df_train = pd.read_csv(train_File)
    df_test = pd.read_csv(test_File)
    df_test['SalePrice'] = 0
    df_concat = pd.concat([df_train,df_test])
    
    df = giveMeWrangledData(df_concat)
    df_prep = preProcessData(df)
    
    X,y = newBoxCoxTranformation(df_prep,'SalePrice',False,False)
    
    score_list = []
    for i in range(max_depth_min,max_depth_max):
        for j in range(n_estimator_min,n_estimator_max,100):
            loop_start = datetime.datetime.now()
            for bytree in [0.25, 0.5, 0.75,1]:
                for bylevel in [0.25, 0.5, 0.75,1]:

                    #reg = XGBRegressor(max_depth=i, n_estimators=j)
                    reg=XGBRegressor(max_depth=i, 
                         n_estimator=j,
                         colsample_bylevel=bylevel,
                         colsample_bytree=bytree,
                         learning_rate=.1,
                         reg_alpha =.5,
                         reg_lambda=.5,
                         n_jobs=n_jobs
                        )

                    #cv = ShuffleSplit(n_splits=20, test_size=random.randint(7,9)/10, random_state=random.randint(1,1000))
                    #cv = ShuffleSplit(n_splits=20, test_size=.5, random_state=1986)
                    #print(datetime.datetime())
                    cross_cv = cross_val_score(reg, X, y,\
                                               cv=forCrossValidationStratifiedShuffleSplit(df_train), \
                                               #cv=cv, \
                                               scoring=crossValidationScoring,n_jobs=n_jobs)
                    #print(" Validat Median Score : ", np.sqrt(np.median(cross_cv) * -1), \
                    #      "Average Score : ", np.sqrt(np.average(cross_cv) * -1) )

                    reg.fit(X,y)
                    training_score = np.sqrt(mean_squared_log_error(np.exp(y), np.exp(reg.predict(X))))
                    #print("Training Score :", training_score)

                    df_test_new, X_test, y_test = checkTheTestFile(reg)
                    reg.fit(X_test,y_test)
                    testing_score = np.sqrt(mean_squared_log_error(np.exp(y), np.exp(reg.predict(X))))
                    #print("Testing Score :", testing_score)
                    print("Scores",(training_score, np.average(cross_cv), testing_score,i,j,bytree, bylevel))
                    score_list.append((training_score, np.average(cross_cv), testing_score,i,j,bytree, bylevel))

        #print("Time for max_depth -",i,"n_estimator -",j," is : ", datetime.datetime.now() - loop_start)
    
    print("Total time for GridSearch : ", datetime.datetime.now() - start_time)
    return score_list

score_list = doGridSearch()
#sorted(score_list,key= lambda x:x[0])

In [ ]:
temp_df = pd.DataFrame(score_list,columns=["training_score",
                                           "validation_score",
                                           "testing_score", 
                                           "max_depth",
                                           "n_estimator",
                                           "bytree",
                                           "bylevel",
                                           #"reg_alpha",
                                           #"reg_lambda"
                                          ])

temp_df.to_csv("GridSearchResults-"+str(datetime.datetime.now().date()))
temp_df

In [ ]:
temp_df[temp_df.testing_score == temp_df.testing_score.min()]

In [ ]:
temp_df[temp_df.validation_score == temp_df.validation_score.min()]

In [ ]:
temp_df[["reg_alpha","validation_score"]].groupby(by="reg_alpha").agg('mean').reset_index()

In [ ]:
temp_df[["reg_lambda","validation_score"]].groupby(by="reg_lambda").agg('median').reset_index()

##### Next Steps:
* re run the grid search and note training score, validation score and testing score. This should not only double check on test score calculation but also gives us the right hyper parameter from the training and validation perspective.
* address the runtime error during scaling or power transform. If boxcox fails attempt a log transformation at least.
* stratify fold testing to check if the training score & validation in the previous exercise still holds goods.
* hyper parameter research for XGBoost
* target variable transformation
* best of best stack approach
* team work stack approach
* XGBoost as the final assesser in best of best stack approach
* XGBoost as the final assesser in the team work starck approach
* 3 layers in stack approach: best of best candidates in the order of their accuracy feeding on input in each case.
* re-assess the dataset

##### Let us talk about the hyper parameter till we get the gridsearch result

* https://xgboost.readthedocs.io/en/latest/parameter.html
* https://www.kaggle.com/dansbecker/xgboost
* https://www.datacamp.com/community/tutorials/xgboost-in-python

* 1 being the max value, let us have half as the value for {'colsample_bytree':0.5, 'colsample_bylevel':0.5}
* learning_rate = 0.05 because we have already using 0.1 so far. It is suggested in the kaggle blog.
* n_jobs= 2 /4 based on the cpu. I guess Kaggle provides 4 cpu machine. I saw the max cpu spike as 400%
* Surprisingly and unnoticed so far that it does the cross validation by itself. so we can leverage the n_estimator to be 1000 and use early_stopping_rounds for our quick turnaround. So our 2 for loops reduced to one :)
